# Financial Filing Agents: Analyzing Corporate Disclosures

In this notebook, we will implement a system of specialized agents for analyzing corporate filings such as 10-Ks, 10-Qs, and annual reports. We'll use the CrewAI framework to create a collaborative agent system that can extract insights from financial documents.

## Objectives
- Build specialized agents for different financial analysis tasks
- Create a coordinated workflow for processing corporate filings
- Extract valuable insights from complex financial documents
- Generate comprehensive analysis reports

In [ ]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
from datetime import datetime
from crewai import Agent, Task, Crew, Process
from langchain.tools import Tool
from langchain.llms import OpenAI

# Set up OpenAI API (you'll need to provide your own API key)
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# Initialize the LLM
llm = OpenAI(temperature=0)

## Step 1: Define Tools for Financial Analysis

First, we'll create a set of tools that our agents can use to process and analyze financial filings.

In [ ]:
# Define a tool for reading financial filings
def read_filing(file_path):
    """Read a financial filing from the given path"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        return f"Error reading file: {str(e)}"

read_filing_tool = Tool(
    name="ReadFiling",
    func=read_filing,
    description="Reads a financial filing document from a given file path"
)

# Define a tool for extracting financial metrics
def extract_metrics(text, metrics_list):
    """Extract specific financial metrics from text"""
    # In a real implementation, this would use regex or NER to extract metrics
    # For now, we'll simulate it with a simple prompt to the LLM
    prompt = f"Extract the following financial metrics from the text: {', '.join(metrics_list)}\n\nText: {text[:5000]}..."
    return llm(prompt)

extract_metrics_tool = Tool(
    name="ExtractMetrics",
    func=extract_metrics,
    description="Extracts specific financial metrics from filing text"
)

# Define a tool for sentiment analysis
def analyze_sentiment(text):
    """Analyze the sentiment of management's discussion"""
    prompt = f"Analyze the sentiment and tone of the following management discussion text. Is it positive, negative, or neutral? What key themes emerge?\n\nText: {text[:5000]}..."
    return llm(prompt)

sentiment_tool = Tool(
    name="AnalyzeSentiment",
    func=analyze_sentiment,
    description="Analyzes the sentiment and tone of management's discussion"
)

# Define a tool for risk factor analysis
def analyze_risk_factors(text):
    """Analyze risk factors mentioned in the filing"""
    prompt = f"Identify and categorize the key risk factors mentioned in the following text:\n\nText: {text[:5000]}..."
    return llm(prompt)

risk_tool = Tool(
    name="AnalyzeRiskFactors",
    func=analyze_risk_factors,
    description="Identifies and categorizes risk factors mentioned in the filing"
)

## Step 2: Create Specialized Agents

Now, we'll define specialized agents for different aspects of financial filing analysis.

In [ ]:
# Document Retrieval Agent
document_agent = Agent(
    role="Financial Document Specialist",
    goal="Retrieve and preprocess financial filings for analysis",
    backstory="""You are an expert in locating and processing financial documents. 
    You know how to navigate complex financial filings and extract the most relevant sections for analysis.""",
    verbose=True,
    allow_delegation=True,
    tools=[read_filing_tool],
    llm=llm
)

# Financial Metrics Agent
metrics_agent = Agent(
    role="Financial Metrics Analyst",
    goal="Extract and analyze key financial metrics from corporate filings",
    backstory="""You are a financial analyst specializing in quantitative metrics. 
    You can extract key financial data points and calculate important ratios to assess company performance.""",
    verbose=True,
    allow_delegation=True,
    tools=[extract_metrics_tool],
    llm=llm
)

# Sentiment Analysis Agent
sentiment_agent = Agent(
    role="Management Sentiment Analyst",
    goal="Analyze the tone and sentiment of management's discussion",
    backstory="""You are an expert in natural language processing and sentiment analysis. 
    You can detect subtle cues in management's language that reveal their outlook and confidence.""",
    verbose=True,
    allow_delegation=True,
    tools=[sentiment_tool],
    llm=llm
)

# Risk Analysis Agent
risk_agent = Agent(
    role="Risk Assessment Specialist",
    goal="Identify and categorize risk factors mentioned in financial filings",
    backstory="""You are a risk management expert who specializes in identifying and evaluating 
    potential risks disclosed in corporate filings. You can categorize risks by type and severity.""",
    verbose=True,
    allow_delegation=True,
    tools=[risk_tool],
    llm=llm
)

# Report Generation Agent
report_agent = Agent(
    role="Financial Report Synthesizer",
    goal="Synthesize findings into a comprehensive analysis report",
    backstory="""You are a financial writer with a talent for distilling complex financial information 
    into clear, insightful reports. You can combine quantitative data and qualitative analysis 
    into actionable investment insights.""",
    verbose=True,
    allow_delegation=True,
    llm=llm
)

## Step 3: Define Tasks for Each Agent

Now we'll define specific tasks for each agent to perform as part of the financial filing analysis workflow.

In [ ]:
# Task for Document Retrieval Agent
retrieve_filing_task = Task(
    description="""Retrieve the financial filing from the specified path and identify its key sections.
    Extract the following sections if present:
    1. Management's Discussion and Analysis (MD&A)
    2. Financial Statements
    3. Risk Factors
    4. Business Overview
    
    Return the content of these sections for further analysis.
    """,
    expected_output="""A dictionary containing the extracted sections from the financial filing, 
    with section names as keys and their content as values.""",
    agent=document_agent
)

# Task for Financial Metrics Agent
extract_metrics_task = Task(
    description="""Analyze the financial statements section to extract the following key metrics:
    1. Revenue
    2. Net Income
    3. Gross Margin
    4. Operating Margin
    5. EPS
    6. Cash and Cash Equivalents
    7. Total Assets
    8. Total Liabilities
    9. Shareholders' Equity
    
    Also calculate the following ratios:
    1. P/E Ratio (if market price is available)
    2. Debt-to-Equity Ratio
    3. Current Ratio
    4. Return on Assets (ROA)
    5. Return on Equity (ROE)
    
    Compare these metrics with the previous year if available.
    """,
    expected_output="""A structured analysis of key financial metrics with year-over-year comparisons 
    and calculated financial ratios.""",
    agent=metrics_agent,
    context=[retrieve_filing_task]
)

# Task for Sentiment Analysis Agent
analyze_sentiment_task = Task(
    description="""Analyze the Management's Discussion and Analysis (MD&A) section to determine:
    1. Overall sentiment (positive, negative, or neutral)
    2. Management's outlook on future performance
    3. Key themes and focus areas
    4. Any notable changes in tone compared to previous statements
    5. Confidence level in their strategies and projections
    
    Look for forward-looking statements and assess the level of optimism or caution.
    """,
    expected_output="""A detailed analysis of management's sentiment, key themes, and outlook, 
    with specific examples from the text supporting the conclusions.""",
    agent=sentiment_agent,
    context=[retrieve_filing_task]
)

# Task for Risk Analysis Agent
analyze_risks_task = Task(
    description="""Analyze the Risk Factors section to:
    1. Identify all significant risks disclosed
    2. Categorize risks (e.g., operational, financial, market, regulatory, technological)
    3. Assess the potential impact of each risk category
    4. Identify any new risks that weren't mentioned in previous filings
    5. Evaluate management's risk mitigation strategies, if mentioned
    
    Prioritize risks based on their potential impact on the company's future performance.
    """,
    expected_output="""A comprehensive risk assessment with categorized risks, their potential impact, 
    and any notable changes from previous disclosures.""",
    agent=risk_agent,
    context=[retrieve_filing_task]
)

# Task for Report Generation Agent
generate_report_task = Task(
    description="""Synthesize the findings from all previous analyses into a comprehensive report that includes:
    1. Executive Summary with key takeaways
    2. Financial Performance Analysis with key metrics and trends
    3. Management Outlook and Sentiment Analysis
    4. Risk Assessment and Mitigation Strategies
    5. Business Overview and Competitive Positioning
    6. Investment Implications and Recommendation
    
    The report should be well-structured, insightful, and provide actionable information for investors.
    """,
    expected_output="""A comprehensive financial analysis report with all the required sections, 
    providing clear insights and investment implications.""",
    agent=report_agent,
    context=[extract_metrics_task, analyze_sentiment_task, analyze_risks_task]
)

## Step 4: Create the Financial Analysis Crew

Now, we'll assemble our agents into a crew with a defined workflow.

In [ ]:
# Create the financial analysis crew
financial_analysis_crew = Crew(
    agents=[document_agent, metrics_agent, sentiment_agent, risk_agent, report_agent],
    tasks=[retrieve_filing_task, extract_metrics_task, analyze_sentiment_task, analyze_risks_task, generate_report_task],
    verbose=2,
    process=Process.sequential  # Tasks will be executed in sequence
)

## Step 5: Run the Crew on a Sample Filing

Let's test our financial analysis crew on a sample filing. You can replace the file path with your own corporate filing.

In [ ]:
# Define the path to your financial filing
# Replace this with the actual path to your financial filing
filing_path = "path/to/your/filing.txt"

# Set the filing path in the task context
retrieve_filing_task.context = f"Financial filing path: {filing_path}"

# Run the crew
result = financial_analysis_crew.kickoff()
print(result)

## Step 6: Customizing the Analysis

The crew can be customized to focus on specific aspects of financial analysis. Here's an example of creating a specialized crew for analyzing future growth prospects.

In [ ]:
# Create a Growth Analysis Agent
growth_agent = Agent(
    role="Growth Prospect Analyst",
    goal="Analyze future growth prospects based on financial filings",
    backstory="""You are an expert in identifying growth opportunities and evaluating management's 
    growth strategies. You can analyze R&D investments, new market entries, and product developments.""",
    verbose=True,
    allow_delegation=True,
    llm=llm
)

# Task for Growth Analysis
analyze_growth_task = Task(
    description="""Analyze the filing to assess future growth prospects:
    1. Identify management's growth strategies
    2. Evaluate R&D investments and initiatives
    3. Analyze new market expansions or product developments
    4. Assess competitive advantages for sustainable growth
    5. Evaluate historical growth rates and future projections
    
    Provide a comprehensive assessment of the company's growth potential.
    """,
    expected_output="""A detailed analysis of growth prospects with evaluation of strategies, 
    investments, and competitive advantages.""",
    agent=growth_agent,
    context=[retrieve_filing_task]
)

# Create a specialized growth analysis crew
growth_analysis_crew = Crew(
    agents=[document_agent, growth_agent, report_agent],
    tasks=[retrieve_filing_task, analyze_growth_task, generate_report_task],
    verbose=2,
    process=Process.sequential
)

## Step 7: Comparative Analysis Across Multiple Filings

We can extend our analysis to compare filings across multiple companies or time periods.

In [ ]:
# Comparative Analysis Agent
comparative_agent = Agent(
    role="Comparative Financial Analyst",
    goal="Compare financial metrics and performance across companies",
    backstory="""You are a comparative analysis expert who excels at benchmarking companies 
    against each other and industry standards. You can identify relative strengths and weaknesses.""",
    verbose=True,
    allow_delegation=True,
    llm=llm
)

# Function to analyze multiple filings
def analyze_multiple_filings(filing_paths, company_names):
    """Run comparative analysis on multiple company filings"""
    all_results = {}
    
    # First, analyze each company individually
    for i, (path, company) in enumerate(zip(filing_paths, company_names)):
        print(f"Analyzing {company}...")
        
        # Update the task context with the current filing path
        retrieve_filing_task.context = f"Financial filing path: {path} for company: {company}"
        
        # Run the crew for this company
        result = financial_analysis_crew.kickoff()
        all_results[company] = result
    
    # Now create a comparative analysis
    comparative_task = Task(
        description=f"""Compare the financial performance, outlook, and risks across these companies: 
        {', '.join(company_names)}. Highlight relative strengths and weaknesses, 
        identify the company with the strongest financial position, best growth prospects, 
        and most favorable risk profile.""",
        expected_output="""A comprehensive comparative analysis highlighting the relative 
        strengths and weaknesses of each company.""",
        agent=comparative_agent,
        context=[str(all_results)]
    )
    
    # Create a crew for the comparative analysis
    comparative_crew = Crew(
        agents=[comparative_agent],
        tasks=[comparative_task],
        verbose=2
    )
    
    # Run the comparative analysis
    comparative_result = comparative_crew.kickoff()
    return comparative_result

# Example usage (uncomment to run)
# filing_paths = ["path/to/company1.txt", "path/to/company2.txt", "path/to/company3.txt"]
# company_names = ["Company A", "Company B", "Company C"]
# comparative_analysis = analyze_multiple_filings(filing_paths, company_names)

## Conclusion

In this notebook, we've built a comprehensive system of financial analysis agents that can collaborate to extract insights from corporate filings. This crew-based approach allows for specialized expertise at each stage of the analysis while maintaining a coordinated workflow.

Key aspects of our implementation:
- Specialized agents with defined roles and expertise
- Task-based workflow with clear dependencies
- Tool integration for specific financial analysis functions
- Flexible architecture that can be customized for different analysis needs
- Comparative analysis capabilities across multiple companies

To use this system with your own financial filings:
1. Place your financial filings in a known directory
2. Update the file paths in the analysis functions
3. Run the appropriate crew based on your analysis needs
4. Extend the system with additional specialized agents as needed